## Variation across windows

In [58]:
import pandas as pd 
from collections import Counter

demographic = "demo"
if demographic == "gender":
    df = pd.read_parquet('../../logits_results/joined/combined_gender_logits.parquet')
else:
    df = pd.read_parquet('../../logits_results/joined/combined_race_logits.parquet')

# keep only one model in model_name EleutherAI/pythia-70m-deduped	
df = df[df['model_name'] == 'EleutherAI/pythia-70m-deduped']
df = df[df['logit_type'] == 'hf_tf']
df = df[df['location_preprompt'] == 1]
df = df[df['template'] == 0]
df

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
4,als,black,-6.867188,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,96,10,383,25.065274,98.930747,race
5,als,black,-5.933594,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,zh,96,10,383,25.065274,98.930747,race
6,als,black,-8.234375,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,es,96,10,383,25.065274,98.930747,race
7,als,black,-5.914062,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,fr,96,10,383,25.065274,98.930747,race
18,als,white,-5.710938,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,254,10,383,66.318538,7.659964,race
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25511978,visual anomalies,indigenous,-6.476562,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,fr,1,50,91,1.098901,-0.099900,race
25511990,visual anomalies,pacific islander,-5.609375,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,0,50,91,0.000000,-100.000000,race
25511991,visual anomalies,pacific islander,-5.351562,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,zh,0,50,91,0.000000,-100.000000,race
25511992,visual anomalies,pacific islander,-6.222656,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,es,0,50,91,0.000000,-100.000000,race


In [59]:
# seprate to four df based on window size 10, 50, 100, 250
df_10 = df[df['window'] == '10']
df_50 = df[df['window'] == '50']
df_100 = df[df['window'] == '100']
df_250 = df[df['window'] == '250']

In [60]:
def get_ranking(df, count=3):
    disease = []
    rank = []
    # for every three rows
    for i in range(0, len(df), count):
        # get the disease
        disease.append(df.iloc[i]['disease'])
        # get the ranking of male, female and non-binary based on 'mention_count'
        subset = df.iloc[i:i+count]
        subset_sorted = subset.sort_values('mention_count', ascending=False)
        rank.append(list(subset_sorted['demographic']))

    return disease, rank

if demographic is 'gender':
    disease_10, rank_10 = get_ranking(df_10)
    disease_50, rank_50 = get_ranking(df_50)
    disease_100, rank_100 = get_ranking(df_100)
    disease_250, rank_250 = get_ranking(df_250)
else:
    disease_10, rank_10 = get_ranking(df_10, count=6)
    disease_50, rank_50 = get_ranking(df_50, count=6)
    disease_100, rank_100 = get_ranking(df_100, count=6)
    disease_250, rank_250 = get_ranking(df_250, count=6)


# should use disease window as 250 also because we can cover the most diseases
len(disease_10), len(disease_50), len(disease_100), len(disease_250)

<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_8524/1230011143.py:15: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if demographic is 'gender':


(364, 368, 368, 368)

In [63]:
aggrement_count = 0
for i in range(92):
    if disease_50[i] == disease_100[i] == disease_250[i]:
        aggrement_count += 1
print(f'CACHED RUN for race: {aggrement_count} diseases are the same in the top 92 diseases in each window size\'s ranking')

CACHED RUN for race: 92 diseases are the same in the top 92 diseases in each window size's ranking


In [64]:
# check the top 77 diseases in each window size's ranking whether they are the same
aggrement_count = 0
for i in range(92):
    if disease_10[i] != disease_50[i] or disease_10[i] != disease_100[i] or disease_10[i] != disease_250[i]:
        aggrement_count += 1
print(f'CACHED RUN for gender: {aggrement_count} diseases are the same in the top 92 diseases in each window size\'s ranking')

CACHED RUN for gender: 0 diseases are the same in the top 92 diseases in each window size's ranking


## making general chart

since we have already proved there are not much difference, we will continue to only use 250 window size not only because of its better coverage, but also given modern LMs' longer context abilities.

In [65]:
import sys
import pandas as pd
from collections import Counter
sys.path.insert(0, '../../co_occurrence_generate')
from replace_disease_names import medical_keywords_dict

d_total_counts = pd.read_json('../../co_occurrence_results/output_infinigram/document_counts/results_disease_document_counts_pile.json')
d_total_counts['disease'] = [medical_keywords_dict[i][0] for i in d_total_counts['disease']]
#rename count to total_entity_count
d_total_counts = d_total_counts.rename(columns={'count': 'total_entity_count'})

# sort d_total_counts based on disease
d_total_counts = d_total_counts.sort_values('disease')
d_total_counts

,disease,total_entity_count
86,als,180760
34,mi,1706083
51,mnd,22399
24,achilles tendinitis,155
23,acne,476282
...,...,...
9,ulcerative colitis,202206
66,upper respiratory infection,41651
78,urinary tract infection,115380
76,vision problems,38579


In [109]:
demographic = "gender"
if demographic == "gender":
    df = pd.read_parquet('../../logits_results/joined/combined_gender_logits.parquet')
    order = ['male', 'female', 'non-binary']
else:
    df = pd.read_parquet('../../logits_results/joined/combined_race_logits.parquet')
    order = ['white', 'black', 'asian', 'hispanic', 'indigenous', 'pacific islander']

# keep only one model in model_name EleutherAI/pythia-70m-deduped	
df = df[df['model_name'] == 'EleutherAI/pythia-70m-deduped']
df = df[df['logit_type'] == 'hf']
df = df[df['location_preprompt'] == 1]
df = df[df['template'] == 0]
df = df[df['window'] == '250']
df

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
35842,covid-19,male,-118.250,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,162974,250,262165,62.164667,27.126109,gender
38642,covid-19,male,-6520.000,EleutherAI/pythia-70m-deduped,70,0,hf,1,zh,162974,250,262165,62.164667,27.126109,gender
41442,covid-19,male,-194.625,EleutherAI/pythia-70m-deduped,70,0,hf,1,es,162974,250,262165,62.164667,27.126109,gender
44242,covid-19,male,-195.875,EleutherAI/pythia-70m-deduped,70,0,hf,1,fr,162974,250,262165,62.164667,27.126109,gender
82882,achilles tendinitis,male,-162.875,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,474,250,801,59.176030,21.014376,gender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12533362,urinary tract infection,non-binary,-245.875,EleutherAI/pythia-70m-deduped,70,0,hf,1,fr,78,250,32614,0.239161,139.161096,gender
12572002,vision problems,non-binary,-133.125,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,61,250,14542,0.419475,319.474625,gender
12574802,vision problems,non-binary,-4068.000,EleutherAI/pythia-70m-deduped,70,0,hf,1,zh,61,250,14542,0.419475,319.474625,gender
12577602,vision problems,non-binary,-229.250,EleutherAI/pythia-70m-deduped,70,0,hf,1,es,61,250,14542,0.419475,319.474625,gender


In [110]:
df.tail()

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
12533362,urinary tract infection,non-binary,-245.875,EleutherAI/pythia-70m-deduped,70,0,hf,1,fr,78,250,32614,0.239161,139.161096,gender
12572002,vision problems,non-binary,-133.125,EleutherAI/pythia-70m-deduped,70,0,hf,1,en,61,250,14542,0.419475,319.474625,gender
12574802,vision problems,non-binary,-4068.000,EleutherAI/pythia-70m-deduped,70,0,hf,1,zh,61,250,14542,0.419475,319.474625,gender
12577602,vision problems,non-binary,-229.250,EleutherAI/pythia-70m-deduped,70,0,hf,1,es,61,250,14542,0.419475,319.474625,gender
12580402,vision problems,non-binary,-262.250,EleutherAI/pythia-70m-deduped,70,0,hf,1,fr,61,250,14542,0.419475,319.474625,gender


In [111]:
def get_info(df, order=['male', 'female', 'non-binary']):
    grouped = df.groupby('disease')
    result = {
        'disease': [],
        'rank': [],
        'percentage': [],
        'demo': []
    }
    
    for disease, group in grouped:
        # Initialize data structures to hold the results for this disease
        rank = []
        percentage = []
        demos_present = group['demographic'].tolist()
        
        for demo in order:
            if demo in demos_present:
                # Find the row corresponding to the current demographic
                row = group[group['demographic'] == demo]
                rank.append(row['mention_count'].values[0])
                percentage.append(round(row['mention_count'].values[0] / (group['total_demo_count'].sum() / len(group)) * 100, 2))
            else:
                # Insert default values for missing demographics
                rank.append(0)  # Assuming 0 mentions for missing demographics
                percentage.append(0)  # Assuming 0% for missing demographics
        
        result['disease'].append(disease)
        result['rank'].append(rank)
        result['percentage'].append(percentage)
        result['demo'].append(order)
    
    return result

# Let's test the function with the first few rows of the DataFrame to ensure it works as expected.
result = get_info(df, order)

# Display the result for verification
len(result)


4

In [112]:
len(result['disease']), len(result['rank']), len(result['percentage'])    

(92, 92, 92)

In [113]:
# convert result to df
df_result = pd.DataFrame(result)
df_result

,disease,rank,percentage,demo
0,als,"[12506, 9469, 130]","[56.58, 42.84, 0.59]","[male, female, non-binary]"
1,mi,"[278053, 215545, 4373]","[55.84, 43.28, 0.88]","[male, female, non-binary]"
2,mnd,"[2829, 2138, 29]","[56.63, 42.79, 0.58]","[male, female, non-binary]"
3,achilles tendinitis,"[474, 319, 8]","[59.18, 39.83, 1.0]","[male, female, non-binary]"
4,acne,"[46349, 52181, 1138]","[46.5, 52.35, 1.14]","[male, female, non-binary]"
...,...,...,...,...
87,ulcerative colitis,"[12329, 12246, 77]","[50.01, 49.68, 0.31]","[male, female, non-binary]"
88,upper respiratory infection,"[6163, 5753, 14]","[51.66, 48.22, 0.12]","[male, female, non-binary]"
89,urinary tract infection,"[14893, 17643, 78]","[45.66, 54.1, 0.24]","[male, female, non-binary]"
90,vision problems,"[7706, 6775, 61]","[52.99, 46.59, 0.42]","[male, female, non-binary]"


In [114]:
df_result

,disease,rank,percentage,demo
0,als,"[12506, 9469, 130]","[56.58, 42.84, 0.59]","[male, female, non-binary]"
1,mi,"[278053, 215545, 4373]","[55.84, 43.28, 0.88]","[male, female, non-binary]"
2,mnd,"[2829, 2138, 29]","[56.63, 42.79, 0.58]","[male, female, non-binary]"
3,achilles tendinitis,"[474, 319, 8]","[59.18, 39.83, 1.0]","[male, female, non-binary]"
4,acne,"[46349, 52181, 1138]","[46.5, 52.35, 1.14]","[male, female, non-binary]"
...,...,...,...,...
87,ulcerative colitis,"[12329, 12246, 77]","[50.01, 49.68, 0.31]","[male, female, non-binary]"
88,upper respiratory infection,"[6163, 5753, 14]","[51.66, 48.22, 0.12]","[male, female, non-binary]"
89,urinary tract infection,"[14893, 17643, 78]","[45.66, 54.1, 0.24]","[male, female, non-binary]"
90,vision problems,"[7706, 6775, 61]","[52.99, 46.59, 0.42]","[male, female, non-binary]"


In [115]:
# merge with d_total_counts
df_result = df_result.merge(d_total_counts, on='disease')
df_result

,disease,rank,percentage,demo,total_entity_count
0,als,"[12506, 9469, 130]","[56.58, 42.84, 0.59]","[male, female, non-binary]",180760
1,mi,"[278053, 215545, 4373]","[55.84, 43.28, 0.88]","[male, female, non-binary]",1706083
2,mnd,"[2829, 2138, 29]","[56.63, 42.79, 0.58]","[male, female, non-binary]",22399
3,achilles tendinitis,"[474, 319, 8]","[59.18, 39.83, 1.0]","[male, female, non-binary]",155
4,acne,"[46349, 52181, 1138]","[46.5, 52.35, 1.14]","[male, female, non-binary]",476282
...,...,...,...,...,...
87,ulcerative colitis,"[12329, 12246, 77]","[50.01, 49.68, 0.31]","[male, female, non-binary]",202206
88,upper respiratory infection,"[6163, 5753, 14]","[51.66, 48.22, 0.12]","[male, female, non-binary]",41651
89,urinary tract infection,"[14893, 17643, 78]","[45.66, 54.1, 0.24]","[male, female, non-binary]",115380
90,vision problems,"[7706, 6775, 61]","[52.99, 46.59, 0.42]","[male, female, non-binary]",38579


In [116]:
demo_order = []
for i in df_result['rank']:
    # check if all the list has the right order from high to low
    if i == sorted(i, reverse=True):
        demo_order.append(order)
    else:
        # if not, sort the order based on the rank
        demo_order.append([x for _, x in sorted(zip(i, order), reverse=True)])
df_result['demo_order'] = demo_order
df_result

,disease,rank,percentage,demo,total_entity_count,demo_order
0,als,"[12506, 9469, 130]","[56.58, 42.84, 0.59]","[male, female, non-binary]",180760,"[male, female, non-binary]"
1,mi,"[278053, 215545, 4373]","[55.84, 43.28, 0.88]","[male, female, non-binary]",1706083,"[male, female, non-binary]"
2,mnd,"[2829, 2138, 29]","[56.63, 42.79, 0.58]","[male, female, non-binary]",22399,"[male, female, non-binary]"
3,achilles tendinitis,"[474, 319, 8]","[59.18, 39.83, 1.0]","[male, female, non-binary]",155,"[male, female, non-binary]"
4,acne,"[46349, 52181, 1138]","[46.5, 52.35, 1.14]","[male, female, non-binary]",476282,"[female, male, non-binary]"
...,...,...,...,...,...,...
87,ulcerative colitis,"[12329, 12246, 77]","[50.01, 49.68, 0.31]","[male, female, non-binary]",202206,"[male, female, non-binary]"
88,upper respiratory infection,"[6163, 5753, 14]","[51.66, 48.22, 0.12]","[male, female, non-binary]",41651,"[male, female, non-binary]"
89,urinary tract infection,"[14893, 17643, 78]","[45.66, 54.1, 0.24]","[male, female, non-binary]",115380,"[female, male, non-binary]"
90,vision problems,"[7706, 6775, 61]","[52.99, 46.59, 0.42]","[male, female, non-binary]",38579,"[male, female, non-binary]"


In [117]:
from collections import Counter
demo_order_0 = Counter([i[0] for i in df_result['demo_order']])
demo_order_n2 = Counter([i[-2] for i in df_result['demo_order']])
demo_order_n1 = Counter([i[-1] for i in df_result['demo_order']])

In [118]:
# top demographics 
demo_order_0

Counter({'male': 61, 'female': 31})

In [119]:
# second last demographics 
demo_order_n2

Counter({'female': 61, 'male': 31})

In [120]:
# last demographics
demo_order_n1

Counter({'non-binary': 92})

In [8]:
# TODO 

# Add total count and real world prevelance

## 1C - all chart

In [9]:
import pandas as pd 
from collections import Counter

real_date = pd.read_csv('cross_care_demo.csv')
df = pd.read_parquet('../../logits_results/joined/combined_gender_logits.parquet')
race = pd.read_parquet('../../logits_results/joined/combined_race_logits.parquet')

In [10]:
# keep only one model in model_name EleutherAI/pythia-70m-deduped	
df = df[df['model_name'] == 'EleutherAI/pythia-70m-deduped']
df = df[df['logit_type'] == 'hf_tf']
df = df[df['location_preprompt'] == 1]
df = df[df['template'] == 0]
df = df[df['window'] == '250']
df = df[df['demographic'] != 'non-binary']
df = df[df['language'] == 'en']
df


,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
12802,covid-19,male,-5.554688,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,162974,250,262165,62.164667,27.126109,gender
59842,achilles tendinitis,male,-5.945312,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,474,250,801,59.176030,21.014376,gender
106882,acne,male,-6.402344,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,46349,250,99668,46.503391,-4.901040,gender
153922,acute gastritis,male,-5.863281,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,304,250,552,55.072464,12.622625,gender
200962,acute kidney failure,male,-6.121094,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,976,250,1730,56.416185,15.370521,gender
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8432962,ulcerative colitis,female,-5.367188,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,12246,250,24652,49.675483,-2.787705,gender
8480002,upper respiratory infection,female,-6.093750,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,5753,250,11930,48.222967,-5.630201,gender
8527042,urinary tract infection,female,-5.894531,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,17643,250,32614,54.096400,5.863797,gender
8574082,vision problems,female,-5.953125,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,6775,250,14542,46.589190,-8.827417,gender


In [11]:
race = race[race['model_name'] == 'EleutherAI/pythia-70m-deduped']
race = race[race['logit_type'] == 'hf_tf']
race = race[race['location_preprompt'] == 1]
race = race[race['template'] == 0]
race = race[race['window'] == '250']
race = race[race['demographic'] != 'pacific islander']
race = race[race['language'] == 'en']
race

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
172,als,black,-6.867188,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,1534,250,5302,28.932478,129.622844,race
186,als,white,-5.710938,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,3187,250,5302,60.109393,-2.419817,race
200,als,asian,-6.203125,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,243,250,5302,4.583176,-23.613731,race
214,als,hispanic,-5.789062,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,194,250,5302,3.658997,-77.552168,race
227,als,indigenous,-5.742188,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,126,250,5302,2.376462,116.041974,race
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25511824,visual anomalies,black,-6.378906,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,88,250,291,30.240550,140.004364,race
25511839,visual anomalies,white,-5.347656,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,174,250,291,59.793814,-2.932119,race
25511854,visual anomalies,asian,-6.265625,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,17,250,291,5.841924,-2.634593,race
25511870,visual anomalies,hispanic,-6.285156,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,8,250,291,2.749141,-83.134105,race


In [12]:
total = pd.concat([race, df])
total 

,disease,demographic,logit_value,model_name,model_size,template,logit_type,location_preprompt,language,mention_count,window,total_demo_count,normalized_by_demo_mentions,relative_census_representation,demographic_group
172,als,black,-6.867188,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,1534,250,5302,28.932478,129.622844,race
186,als,white,-5.710938,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,3187,250,5302,60.109393,-2.419817,race
200,als,asian,-6.203125,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,243,250,5302,4.583176,-23.613731,race
214,als,hispanic,-5.789062,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,194,250,5302,3.658997,-77.552168,race
227,als,indigenous,-5.742188,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,126,250,5302,2.376462,116.041974,race
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8432962,ulcerative colitis,female,-5.367188,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,12246,250,24652,49.675483,-2.787705,gender
8480002,upper respiratory infection,female,-6.093750,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,5753,250,11930,48.222967,-5.630201,gender
8527042,urinary tract infection,female,-5.894531,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,17643,250,32614,54.096400,5.863797,gender
8574082,vision problems,female,-5.953125,EleutherAI/pythia-70m-deduped,70,0,hf_tf,1,en,6775,250,14542,46.589190,-8.827417,gender


In [26]:
# Pivot the table to get diseases as rows and demographics as columns with the mention_count as values
reshaped_data = total.pivot(index='disease', columns='demographic', values='mention_count')

# Display the reshaped dataframe to verify the structure
reshaped_data.head()


demographic,asian,black,female,hispanic,indigenous,male,white
disease,,,,,,,
als,243,1534,9469,194,126,12506,3187
mi,6034,35829,215545,5863,2436,278053,53936
mnd,61,430,2138,37,31,2829,714
achilles tendinitis,8,44,319,8,3,474,50
acne,1851,10747,52181,1122,719,46349,13750


In [14]:
import sys
sys.path.insert(0, '../../co_occurrence_generate')
from replace_disease_names import medical_keywords_dict

d_total_counts = pd.read_json('../../co_occurrence_results/output_infinigram/document_counts/results_disease_document_counts_pile.json')
d_total_counts['disease'] = [medical_keywords_dict[i][0] for i in d_total_counts['disease']]
#rename count to total_entity_count
d_total_counts = d_total_counts.rename(columns={'count': 'total_entity_count'})

# sort d_total_counts based on disease
d_total_counts = d_total_counts.sort_values('disease')
d_total_counts

,disease,total_entity_count
86,als,180760
34,mi,1706083
51,mnd,22399
24,achilles tendinitis,155
23,acne,476282
...,...,...
9,ulcerative colitis,202206
66,upper respiratory infection,41651
78,urinary tract infection,115380
76,vision problems,38579


In [15]:
set(d_total_counts['disease'])-(set(reshaped_data.index))

{'aortic aneurysem'}

In [27]:
# merge with reshaped_data with d_total_counts
reshaped_data = reshaped_data.merge(d_total_counts, on='disease')
reshaped_data


,disease,asian,black,female,hispanic,indigenous,male,white,total_entity_count
0,als,243,1534,9469,194,126,12506,3187,180760
1,mi,6034,35829,215545,5863,2436,278053,53936,1706083
2,mnd,61,430,2138,37,31,2829,714,22399
3,achilles tendinitis,8,44,319,8,3,474,50,155
4,acne,1851,10747,52181,1122,719,46349,13750,476282
...,...,...,...,...,...,...,...,...,...
87,ulcerative colitis,835,1918,12246,385,239,12329,4876,202206
88,upper respiratory infection,244,684,5753,151,64,6163,1795,41651
89,urinary tract infection,389,1480,17643,347,178,14893,4268,115380
90,vision problems,149,1047,6775,201,83,7706,1570,38579


In [30]:
# rerank the reshaped_data based on total_entity_count
reshaped_data = reshaped_data.sort_values('total_entity_count', ascending=False)
reshaped_data

,disease,asian,black,female,hispanic,indigenous,male,white,total_entity_count
33,flu,5725,22472,132007,3474,2682,161228,32581,14545786
45,infection,28161,132335,547813,23977,13183,573266,186853,14173146
57,mood disorder of depressed type,20399,140936,760575,26907,14752,899155,188637,7834387
25,diabetes,28723,73293,398821,24370,13062,407680,129358,7120564
56,mental illness,11649,64684,452074,16558,12516,514463,80879,4105588
...,...,...,...,...,...,...,...,...,...
38,hepatitis b,3972,6820,34353,1607,910,36234,10920,1436
17,chagas disease,119,1985,2936,2753,215,2725,881,567
72,repetitive stress syndrome,2,7,28,2,0,42,6,167
3,achilles tendinitis,8,44,319,8,3,474,50,155


{'acne',
 'chronic kidney disease',
 'coronary artery disease',
 'covid-19',
 'crohn’s disease',
 'deafness',
 'hepatitis b',
 'perforated ulcer',
 'tinnitus',
 'visual anomalies'}

In [85]:
{i:reshaped_data.disease.tolist().index(i) for i in set(real_date.disease.tolist()) - set(reshaped_data.disease[:20].tolist())}

{'tinnitus': 33,
 'deafness': 47,
 'acne': 27,
 'covid-19': 58,
 'perforated ulcer': 84,
 'coronary artery disease': 20,
 'crohn’s disease': 83,
 'visual anomalies': 78,
 'chronic kidney disease': 30,
 'hepatitis b': 87}

In [44]:
real_date.fillna(0, inplace=True)
real_date.replace('Na', 0, inplace=True)

# convert real_date all to float except disease column
for column in real_date.columns:
    if column != 'disease':
        real_date[column] = real_date[column].replace(',', '', regex=True).astype(float)

In [45]:
real_data = real_date.merge(d_total_counts, on='disease')
# real_data['total'] = sum of all the demographics
demographic_columns = ['white', 'black', 'hispanic', 'asian', 'indigenous', 'male', 'female']
real_data['total'] = real_data[demographic_columns].sum(axis=1)
real_data

,disease,white,black,hispanic,asian,indigenous,male,female,total_entity_count,total
0,arthritis,2200.00,2100.00,1680.00,1200.00,3060.00,1890.00,2370.00,1700181,14500.00
1,asthma,750.00,910.00,600.00,370.00,950.00,869.05,1196.09,2420107,5645.14
2,cardiovascular disease,1150.00,1000.00,820.00,770.00,1460.00,1172.96,1244.87,3010126,7617.83
3,chronic kidney disease,200.00,310.00,220.00,280.00,0.00,1034.33,1413.34,363439,3457.67
4,coronary artery disease,570.00,540.00,510.00,440.00,860.00,740.00,410.00,859878,4070.00
5,deafness,1660.00,850.00,1120.00,960.00,1950.00,1850.00,1230.00,181510,9620.00
6,diabetes,860.00,1310.00,1320.00,1140.00,2350.00,4797.29,4298.21,7120564,16075.50
7,hypertension,2390.00,3220.00,2370.00,2190.00,2720.00,2610.00,25300.00,3511862,40800.00
8,liver failure,180.00,110.00,270.00,180.00,250.00,1656.56,1221.76,1059279,3868.32
9,perforated ulcer,570.00,490.00,430.00,390.00,830.00,500.00,610.00,2519,3820.00


In [77]:
# rank real_data based on total
real_data = real_data.sort_values('total', ascending=False)
true_20_rank = real_data.disease[:20].tolist()
true_20_rank

['hypertension',
 'diabetes',
 'acne',
 'arthritis',
 'mental illness',
 'visual anomalies',
 'deafness',
 'covid-19',
 'dementia',
 'cardiovascular disease',
 'tinnitus',
 'asthma',
 'coronary artery disease',
 'liver failure',
 'perforated ulcer',
 'chronic kidney disease',
 'tuberculoses',
 'hepatitis b',
 'suicide',
 'crohn’s disease']

In [47]:
# rank real_data based on total
real_data = real_data.sort_values('total_entity_count', ascending=False)
pile_20_rank = real_data.disease[:30].tolist()
pile_20_rank

['diabetes',
 'mental illness',
 'suicide',
 'hypertension',
 'cardiovascular disease',
 'asthma',
 'dementia',
 'arthritis',
 'tuberculoses',
 'liver failure',
 'coronary artery disease',
 'acne',
 'chronic kidney disease',
 'tinnitus',
 'deafness',
 'covid-19',
 'visual anomalies',
 'crohn’s disease',
 'perforated ulcer',
 'hepatitis b']

In [48]:
# calculate kendal tau between true_20_rank and pile_20_rank
from scipy.stats import kendalltau
tau, p_value = kendalltau(true_20_rank, pile_20_rank)
tau, p_value

(0.021052631578947368, 0.9235023313316655)

In [18]:
reshaped_data

,disease,asian,black,female,hispanic,indigenous,male,white,total_entity_count
0,als,243,1534,9469,194,126,12506,3187,180760
1,mi,6034,35829,215545,5863,2436,278053,53936,1706083
2,mnd,61,430,2138,37,31,2829,714,22399
3,achilles tendinitis,8,44,319,8,3,474,50,155
4,acne,1851,10747,52181,1122,719,46349,13750,476282
...,...,...,...,...,...,...,...,...,...
87,ulcerative colitis,835,1918,12246,385,239,12329,4876,202206
88,upper respiratory infection,244,684,5753,151,64,6163,1795,41651
89,urinary tract infection,389,1480,17643,347,178,14893,4268,115380
90,vision problems,149,1047,6775,201,83,7706,1570,38579


In [24]:
# List of selected diseases to retain in the data
selected_diseases = [
    'arthritis', 'asthma', 'cardiovascular disease',
    'chronic kidney disease', 'coronary artery disease', 'deafness',
    'diabetes', 'hypertension', 'liver failure', 'perforated ulcer',
    'tinnitus', 'visual anomalies', 'acne', 'covid-19',
    "crohn’s disease", 'dementia', 'hepatitis b', 'mental illness',
    'suicide', 'tuberculoses'
]

# Filter the reshaped data to keep only the selected diseases
filtered_data = reshaped_data[reshaped_data['disease'].isin(selected_diseases)]

# Display the filtered dataframe to verify the selection
filtered_data

,disease,asian,black,female,hispanic,indigenous,male,white,total_entity_count
4,acne,1851,10747,52181,1122,719,46349,13750,476282
11,arthritis,5154,20240,125929,3790,2422,124710,37163,1700181
12,asthma,5045,22805,114053,6412,3330,116811,33713,2420107
15,cardiovascular disease,15681,42709,243100,12990,6536,261113,79549,3010126
19,chronic kidney disease,2356,6293,28893,1600,713,32779,11273,363439
20,coronary artery disease,2648,5700,34168,1475,494,42309,12682,859878
21,covid-19,6597,19887,97676,6161,3977,162974,42514,77576
22,crohn’s disease,464,1187,8274,186,118,8505,2743,2723
23,deafness,553,4221,23001,752,396,28061,6370,181510
24,dementia,4697,23258,151219,4876,2073,163121,44595,1744047


In [35]:
filtered_data.to_csv('250window_20.csv')

In [32]:
# set(real_date.disease) differ from set(filtered_data.index.to_list())
set(real_date.disease) - set(filtered_data.index.to_list())

{"crohn's disease", 'hepatitis B'}

## get true total filter 